In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
# 현재경로 확인
os.getcwd()

'C:\\Users\\Uijeong'

In [4]:
# 데이터 불러오기
data = pd.read_csv(r"C:\Users\Uijeong\Desktop\05-11--machine-learning\Part 05~11) Machine Learning\08. Ensemble 기법의 종류와 원리\Data\kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [5]:
'''
id: 집 고유아이디
date: 집이 팔린 날짜 
price: 집 가격 (타겟변수)
bedrooms: 주택 당 침실 개수
bathrooms: 주택 당 화장실 개수
floors: 전체 층 개수
waterfront: 해변이 보이는지 (0, 1)
condition: 집 청소상태 (1~5)
grade: King County grading system 으로 인한 평점 (1~13)
yr_built: 집이 지어진 년도
yr_renovated: 집이 리모델링 된 년도
zipcode: 우편번호
lat: 위도
long: 경도
'''

'\nid: 집 고유아이디\ndate: 집이 팔린 날짜 \nprice: 집 가격 (타겟변수)\nbedrooms: 주택 당 침실 개수\nbathrooms: 주택 당 화장실 개수\nfloors: 전체 층 개수\nwaterfront: 해변이 보이는지 (0, 1)\ncondition: 집 청소상태 (1~5)\ngrade: King County grading system 으로 인한 평점 (1~13)\nyr_built: 집이 지어진 년도\nyr_renovated: 집이 리모델링 된 년도\nzipcode: 우편번호\nlat: 위도\nlong: 경도\n'

In [6]:
nCar = data.shape[0] # 데이터 개수
nVar = data.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 21613 nVar: 14


In [6]:
ncar=data.shape[0]
nvar=data.shape[1]
print(ncar)
print(nvar)

21613
14


## 의미가 없다고 판단되는 변수 제거

In [7]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # id, date, zipcode, lat, long  제거

## 범주형 변수를 이진형 변수로 변환
- 범주형 변수는 waterfront 컬럼 뿐이며, 이진 분류이기 때문에 0, 1로 표현한다.
- 데이터에서 0, 1로 표현되어 있으므로 과정 생략

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [8]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행 (특성만 따로 저장)
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증

In [11]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sm_train_x=sm.add_constant(train_x,has_constant='add')
sm_model=sm.OLS(train_y,sm_train_x)
fitted_sm_model=sm_model.fit()
fitted_sm_model.summary()
#R_squared: y의 총 변동성 중 x의 변동성이 영향을 미치는 정도
#p_value 확인 , bedroom하고 bathroom 아마도 다중공산성

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Thu, 11 Feb 2021   Prob (F-statistic):               0.00
Time:                        22:17:26   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
sm_test_x=sm.add_constant(test_x, has_constant='add')
sm_model_predict=fitted_sm_model.predict(sm_test_x)

In [13]:
sqrt(mean_squared_error(sm_model_predict,test_y)) #RMSE

239804.29670858165

## Bagging 결과 vs. 일반적인 결과

In [20]:
bagging_predict_result=[]
for _ in range(10):
    data_index=[data_index for data_index in range (train_x.shape[0])]
    random_data_index=np.random.choice(data_index,train_x.shape[0]) #복원추출
    sm_train_x=train_x.iloc[random_data_index,]
    sm_train_y=train_y.iloc[random_data_index,]
    sm_train_x=sm.add_constant(sm_train_x,has_constant='add')
    sm_model=sm.OLS(sm_train_y, sm_train_x)
    fitted_sm_model=sm_model.fit()
    pred=fitted_sm_model.predict(sm_test_x)
    bagging_predict_result.append(pred)
    

In [21]:
bagging_predict=[]
for lst2_index in range (test_x.shape[0]): #test data의 개수만큼 
    temp_predict=[]
    for lst_index in range (len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[lst_index].values[lst2_index]) #각 리스트의 해당 인덱스의 값 추가
    bagging_predict.append(np.mean(temp_predict))

In [22]:
temp_predict

[442087.76009420044,
 439379.6612492322,
 427179.09858439886,
 438186.96826080506,
 434392.04001912184,
 435299.31058491394,
 441949.0892018179,
 440644.1693952125,
 432534.60909474146,
 431055.04686161503]

In [23]:
sqrt(mean_squared_error(bagging_predict,test_y)) #RMSE

239859.77546937333

## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (Scikit-Learn)

In [25]:
regression_model = LinearRegression() # 선형 회귀 모형
linear_model1 = regression_model.fit(train_x, train_y) # 학습 데이터를 선형 회귀 모형에 적합
predict1 = linear_model1.predict(test_x) # 학습된 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 239804.2967085815


## Bagging을 이용하여 선형 회귀 모형에 적합 후 평가 (Sampling 10번)

In [26]:
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = regression_model, # 선형회귀모형
                                 n_estimators = 5, # 5번 샘플링
                                 verbose = 1) # 학습 과정 표시
linear_model2 = bagging_model.fit(train_x, train_y) # 학습 진행
predict2 = linear_model2.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 239833.64056043772


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## 그렇다면 Sampling을 많이 해보자!

In [11]:
bagging_model2 = BaggingRegressor(base_estimator = regression_model, # 선형 회귀모형
                                  n_estimators = 30, # 30번 샘플링
                                  verbose = 1) # 학습 과정 표시
linear_model3 = bagging_model2.fit(train_x, train_y) # 학습 진행
predict3 = linear_model3.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

RMSE: 239802.25459625298


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## 학습 데이터를 의사결정나무모형에 적합 후 평가 데이터로 검증

In [37]:
from sklearn.tree import DecisionTreeRegressor
decision_tree_model = DecisionTreeRegressor() # 의사결정나무 모형
tree_model1 = decision_tree_model.fit(train_x, train_y) # 학습 데이터를 의사결정나무 모형에 적합
predict1 = tree_model1.predict(test_x) # 학습된 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과
#RMSE는 값이 작을수록 성능이 좋은 것

RMSE: 297060.55730945733


In [38]:
bagging_predict_result=[]
for _ in range(10):
    data_index=[data_index for data_index in range (train_x.shape[0])]
    random_data_index=np.random.choice(data_index,train_x.shape[0]) #복원추출
    sm_train_x=train_x.iloc[random_data_index,]
    sm_train_y=train_y.iloc[random_data_index,]
    decision_tree_model = DecisionTreeRegressor() # 의사결정나무 모형
    tree_model1 = decision_tree_model.fit(train_x, train_y) # 학습 데이터를 의사결정나무 모형에 적합
    predict1 = tree_model1.predict(test_x)
    bagging_predict_result.append(predict1)
    print(sqrt(mean_squared_error(predict1,test_y)))

    #tree모델은 과적합이 쉽게 일어나기 때문에 성능이 엄청 좋게 나오지 않음

297210.2662015311
296034.23911489826
298787.83849386114
286717.61643394455
289076.3750148254
287565.13685568544
296856.91558401217
298083.9140903002
288518.3666741056
290417.3952741628


In [39]:
bagging_predict=[]
for lst2_index in range (test_x.shape[0]): #test data의 개수만큼 
    temp_predict=[]
    for lst_index in range (len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) #각 리스트의 해당 인덱스의 값 추가
    bagging_predict.append(np.mean(temp_predict))

In [40]:
sqrt(mean_squared_error(bagging_predict,test_y))
# bagging 하면 좋아짐 -> 앙상블을 통해 많이 좋아짐. (다양한 의견 수렴을 위해 tree 기반 모델 사용)

288471.0009907292

## Bagging 을 이용하여 의사결정나무모형에 적합 후 평가 (Sampling 10번)

In [41]:
bagging_decision_tree_model1 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 5, # 5번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model2 = bagging_decision_tree_model1.fit(train_x, train_y) # 학습 진행
predict2 = tree_model2.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 251232.83025756502


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [42]:
bagging_decision_tree_model2 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 30, # 30번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model3 = bagging_decision_tree_model2.fit(train_x, train_y) # 학습 진행
predict3 = tree_model3.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE: 236714.68715092994


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
